<a href="https://colab.research.google.com/github/kiwi-Shruti-Bhat/Model/blob/main/BMI_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Objective : 

Give a human image , identify height and weight and BMI of the individual. 

## Solution :

The solution has been inspired by the paper [FACE2BMI](https://arxiv.org/pdf/1703.03156.pdf). This paper shows that face can be an important estimator of your height and weight and hence the BMI. The dataset quoted in the papaer is not publicly available hence i could not use the mentioned dataset for my work. I gathered data from internet manualyy. I have used almost similar technique in which i have used the celebrity's face to estimate their height,weight and BMI. 

## Solution workflow

 > Image ==> Face ==>  Face embedding ==> Height
 
 > Image ==> Face ==>  Face embedding ==> Weight
 
 > Image ==> Face ==>  Face embedding ==> BMI

## Data  Collection and Augmentation

The dataset consists of 5-20 images of bollywood celeberities. The various images has been taken under different conditions so that we have enough variations.
The images that has been taken is downloaded using google serach engine results. Care has been taken care that all the images are latest. The label for each celeberity has been noted from the publicly available forums.


In [ ]:
label_file = "./BMI data - Sheet1.csv"

In [ ]:
import pandas as pd

In [ ]:
profile_df = pd.read_csv(label_file)

In [ ]:
profile_df

,id,UID,name,height,weight,BMI
0,1,akshay,akshay kumar,1.78,80,25.249337
1,2,ja,john abraham,1.82,94,28.378215
2,3,varun,varun dhawan,1.68,78,27.636054
3,4,tiger,Tiger shroff,1.75,72,23.510204
4,5,salman,Salman khan,1.74,75,24.772097
5,6,srk,Shahrukh khan,1.73,75,25.059307
6,7,ayushman,ayushman khurana,1.68,72,25.510204
7,8,vikky,vicky kaushal,1.83,80,23.888441
8,9,rajkumar,rajkumar rao,1.70,72,24.913495
9,10,nawaz,nawazuddin,1.68,60,21.258503


## How is BMI calculated?
```
BMI = (Weight in Kg) / ((Height in Meters) ^ 2)
```


All the training images are kept in the following directory

In [ ]:
data_folder = "height_weight"

In [ ]:
from glob import glob
all_files = glob(data_folder+"/*")

all_jpgs = sorted([img for img in all_files if ".jpg" in img or ".jpeg" in img or "JPG" in img])

print("Total {} photos ".format(len(all_jpgs)))

Total 226 photos 


In [ ]:
from pathlib import Path as p

In [ ]:
def get_index_of_digit(string):
    import re
    match = re.search("\d", p(string).stem)
    return match.start(0)

In [ ]:
id_path = [(p(images).stem[:(get_index_of_digit(p(images).stem))],images) for  images in all_jpgs ]

In [ ]:
image_df = pd.DataFrame(id_path,columns=['UID','path'])

In [ ]:
data_df = image_df.merge(profile_df) ## merged the training images with their profile

In [ ]:
data_df

,UID,path,id,name,height,weight,BMI
0,akshay,height_weight/akshay1.jpeg,1,akshay kumar,1.78,80,25.249337
1,akshay,height_weight/akshay10.jpg,1,akshay kumar,1.78,80,25.249337
2,akshay,height_weight/akshay11.jpg,1,akshay kumar,1.78,80,25.249337
3,akshay,height_weight/akshay12.jpg,1,akshay kumar,1.78,80,25.249337
4,akshay,height_weight/akshay13.jpg,1,akshay kumar,1.78,80,25.249337
...,...,...,...,...,...,...,...
221,vikky,height_weight/vikky5.jpg,8,vicky kaushal,1.83,80,23.888441
222,vikky,height_weight/vikky6.jpg,8,vicky kaushal,1.83,80,23.888441
223,vikky,height_weight/vikky7.jpg,8,vicky kaushal,1.83,80,23.888441
224,vikky,height_weight/vikky8.jpg,8,vicky kaushal,1.83,80,23.888441


## Extract face embedding using facenet pretrained architecture

In [ ]:
!pip3 install face_recognition

     |████████████████████████████████| 100.1 MB 26 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566185 sha256=a40df89bfdf7d0d067a2d8887677cb8b8cc367f3f85a4a8db714c73c1fa15067
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [ ]:
import face_recognition
import numpy as np

In [ ]:
def get_face_encoding(image_path):
    print(image_path)
    picture_of_me = face_recognition.load_image_file(image_path)
    my_face_encoding = face_recognition.face_encodings(picture_of_me)
    if not my_face_encoding:
        print("no face found !!!")
        return np.zeros(128).tolist()
    return my_face_encoding[0].tolist()

In [ ]:
def get_face_test(url) :
    response = urllib.request.urlopen(url)
    picture_of_me = face_recognition.load_image_file(response)
    my_face_encoding = face_recognition.face_encodings(picture_of_me)
    if not my_face_encoding:
        print("no face found !!!")
        return np.zeros(128).tolist()
    return my_face_encoding[0].tolist()

In [ ]:
all_faces = []

In [ ]:
for images in data_df.path:
    face_enc = get_face_encoding(images)
    all_faces.append(face_enc)

height_weight/akshay1.jpeg
height_weight/akshay10.jpg
height_weight/akshay11.jpg
height_weight/akshay12.jpg
height_weight/akshay13.jpg
height_weight/akshay14.jpg
height_weight/akshay15.jpg
height_weight/akshay16.jpg
height_weight/akshay17.jpg
height_weight/akshay18.jpg
height_weight/akshay19.jpg
height_weight/akshay2.jpeg
no face found !!!
height_weight/akshay20.jpg
height_weight/akshay3.jpg
height_weight/akshay4.jpg
height_weight/akshay5.jpg
height_weight/akshay6.jpg
height_weight/akshay7.jpg
height_weight/akshay8.jpg
height_weight/akshay9.jpg
height_weight/amir1.jpg
height_weight/amir10.jpg
height_weight/amir11.jpg
height_weight/amir12.jpg
height_weight/amir13.jpg
height_weight/amir14.jpeg
height_weight/amir15.jpg
height_weight/amir2.jpg
height_weight/amir3.jpg
height_weight/amir4.jpeg
no face found !!!
height_weight/amir5.jpg
height_weight/amir7.jpeg
height_weight/amir8.jpg
height_weight/amir9.jpg
height_weight/anupam2.jpg
height_weight/anupam3.jpg
height_weight/anupam4.jpg
height_w

In [ ]:
X = np.array(all_faces) ## This is the training data matrix

In [ ]:

y_height = data_df.height.values ## all labels
y_weight = data_df.weight.values
y_BMI = data_df.BMI.values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_height_train, y_height_test, y_weight_train, y_weight_test ,y_BMI_train, y_BMI_test = train_test_split(X, y_height,y_weight,y_BMI, random_state=1)

## Metric to check the goodness of fit

In [ ]:
def report_goodness(model,X_test,y_test,predictor_log=True):
    # Make predictions using the testing set
    y_pred = model.predict(X_test)
    y_true = y_test
    if predictor_log:
        y_true = np.log(y_test)
    # The coefficients
    # The mean squared error
    print("Mean squared error: %.2f"      % mean_squared_error(y_true, y_pred))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(y_true, y_pred))
    
    errors = abs(y_pred - y_true)
    mape = 100 * np.mean(errors / y_true)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))

## Model selection 

In [ ]:
from sklearn.kernel_ridge import KernelRidge
from sklearn import  linear_model
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

## Kernel Ridge

## Height

In [ ]:
model_height = KernelRidge(kernel='rbf', gamma=0.21,alpha=0.0017)

In [ ]:
model_height = model_height.fit(X_train,np.log(y_height_train))

report_goodness(model_height,X_test,y_height_test)

Mean squared error: 0.00
Variance score: 0.80
Model Performance
Average Error: 0.0132 degrees.
Accuracy = 97.52%.


## Weight

In [ ]:
model_weight = KernelRidge(kernel='rbf', gamma=0.21,alpha=0.0017)

In [ ]:
model_weight = model_weight.fit(X_train,np.log(y_weight_train))

report_goodness(model_weight,X_test,y_weight_test)

Mean squared error: 0.00
Variance score: 0.86
Model Performance
Average Error: 0.0458 degrees.
Accuracy = 98.92%.


## BMI

In [ ]:
model_BMI = KernelRidge(kernel='rbf', gamma=0.21,alpha=0.0017)

In [ ]:
model_BMI = model_BMI.fit(X_train,np.log(y_BMI_train))

report_goodness(model_BMI,X_test,y_BMI_test)

Mean squared error: 0.00
Variance score: 0.88
Model Performance
Average Error: 0.0311 degrees.
Accuracy = 99.01%.


In [ ]:
report_goodness(model_BMI,X_test,y_BMI_test)

Mean squared error: 0.00
Variance score: 0.88
Model Performance
Average Error: 0.0311 degrees.
Accuracy = 99.01%.


## Conclusion : 

We found that kernelised Ridge regression outperformed all the models in terms of the mean squared error and expalined variance

## save all models 

In [ ]:
import joblib

# save the model to disk

In [ ]:
height_model = 'weight_predictor.model'
weight_model = 'height_predictor.model'
bmi_model = 'bmi_predictor.model'
joblib.dump(model_height, height_model)
joblib.dump(model_weight, weight_model)
joblib.dump(model_BMI, bmi_model)

['bmi_predictor.model']

# load the model from disk

In [ ]:
height_model = joblib.load(height_model)
weight_model = joblib.load(weight_model)
bmi_model = joblib.load(bmi_model)

## test 

In [ ]:
def predict_height_width_BMI(test_image,height_model,weight_model,bmi_model):
    test_array = np.expand_dims(np.array(get_face_test(test_image)),axis=0)
    height = np.asscalar(np.exp(height_model.predict(test_array)))
    weight = np.asscalar(np.exp(weight_model.predict(test_array)))
    bmi = np.asscalar(np.exp(bmi_model.predict(test_array)))
    return {'height':height,'weight':weight,'bmi':bmi}
    

## prediction on test images

In [ ]:
from IPython.display import Image
from PIL import Image
import requests
from io import BytesIO

import urllib.request

In [ ]:
url = "https://scontent-lga3-2.cdninstagram.com/v/t51.2885-15/sh0.08/e35/c0.180.1440.1440a/s640x640/84287768_128018378735197_4647999861785324604_n.jpg?_nc_ht=scontent-lga3-2.cdninstagram.com&_nc_cat=101&_nc_ohc=JTmO0eYPVZIAX9b43BA&edm=ALLxIhEBAAAA&ccb=7-4&oh=b456104b05ded883f5ef242a9da0f17d&oe=61A424CB&_nc_sid=b2a057"

In [ ]:
print(predict_height_width_BMI(url,height_model,weight_model,bmi_model))

{'height': 1.6537956536748297, 'weight': 52.03463494747251, 'bmi': 21.79184624102154}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  """


## Conclusion

Face is almost able to predict BMI correctly. There are places where it is going wrong for example if you predict heigt of really tall celebrity like rampal.
This behavior is acceptable since we have not trained the model on such tall celebrities. Other,issue that i noticed is the face to weight tagging is not that much appropriate since the faces are from diffrent times but the weight is always the latest.
We can get even better model if we have these mappings right.

Also we can improve on height prediction if we model the torso alongwith the face. However , we need to get torso to correct height labeled data for this. In the absence of such data we can rely on the face as height estimator.